In [1]:
import pandas as pd
import numpy as np
import math
import tensorflow as tf
from tensorflow.keras import backend as K

df = pd.read_csv('G:/Deep_Ha/kidney.csv')

x_data = df.loc[:,['age','sex']]
y_data = df.loc[:,['time','status']]
x_data['sex'] = x_data.pop('sex') - 1

In [2]:
df.head()

,id,time,status,age,sex,disease,frail
0,1,8,1,28,1,Other,2.3
1,1,16,1,28,1,Other,2.3
2,2,23,1,48,2,GN,1.9
3,2,13,0,48,2,GN,1.9
4,3,22,1,32,1,Other,1.2


In [3]:
#pip install lifelines

In [4]:
from lifelines import CoxPHFitter
cph = CoxPHFitter()
cph.fit(df, duration_col='time', event_col='status', formula="age + sex")

cph.print_summary()  # access the individual results using cph.summary


<lifelines.CoxPHFitter: fitted with 76 total observations, 18 right-censored observations>
             duration col = 'time'
                event col = 'status'
      baseline estimation = breslow
   number of observations = 76
number of events observed = 58
   partial log-likelihood = -184.34
         time fit was run = 2022-07-15 04:54:26 UTC

---
            coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                         
age         0.00       1.00       0.01            -0.02             0.02                 0.98                 1.02
sex        -0.83       0.44       0.30            -1.42            -0.24                 0.24                 0.78

            cmp to     z    p   -log2(p)
covariate                               
age           0.00  0.22 0.83       0.28
sex           0.00 -2.77 0.01       7.50
---
Concordance = 0.66
Partial AIC = 372.69
log-likelihood ratio test = 7.12 on 2 df
-log2(p) of ll-ratio test = 5.13

In [5]:
pred=cph.predict_partial_hazard(x_data)
pred

0     4.090028
1     4.090028
2     1.858633
3     1.858633
4     4.123405
        ...   
71    1.836111
72    1.873801
73    1.873801
74    4.364799
75    4.364799
Length: 76, dtype: float64

In [6]:
from lifelines.utils import concordance_index
time = df.loc[:,['time']]
status = df.loc[:,['status']]
concordance_index(time, -pred, event_observed=status)

0.6621827411167512

In [7]:
concordance_index(time, -np.exp(pred), event_observed=status)

0.6621827411167512